# Notebook for cleaning `OBSERVATION TABLE`

In [1]:
import pandas as pd
import numpy as np
 
from sqlalchemy import create_engine,text
user = 'root'
passwords = '7003890541'
host = 'localhost'
port = 3306
database = 'synthea'

engine = create_engine(f"mysql+pymysql://{user}:{passwords}@{host}:{port}/{database}")
def sql(query , engine = engine):
    return pd.read_sql(query,engine)


def select(table_name):
    return sql(f'select * from {table_name} limit 5')


def run(query):
    with engine.connect() as con:
        con.execute(text(query))
        con.commit()

In [2]:
select('observations')

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,8302-2,Body Height,193.3,cm,numeric
1,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,2.0,{score},numeric
2,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,29463-7,Body Weight,87.8,kg,numeric
3,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,39156-5,Body Mass Index,23.5,kg/m2,numeric
4,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,8462-4,Diastolic Blood Pressure,82.0,mm[Hg],numeric


In [3]:
query = '''
    alter table observations
    drop column code;
'''
run(query)

In [4]:
query = '''
    select column_name from information_schema.columns
    where table_name = 'observations' and
    table_schema = database();
'''
sql(query)
observations_columns = [c[0] for c in sql(query).values.tolist()]

In [5]:
for col in observations_columns:
    query = f'''
        alter table observations
        rename column {col} to {col.lower()}
    '''
    run(query)

In [6]:
for col in observations_columns:
    query = f'''
        update observations
        set {col} = lower({col})
    '''
    run(query)

In [7]:
query = """
UPDATE observations
SET date = STR_TO_DATE(
    TRIM(REPLACE(REPLACE(REPLACE(date, '"', ''), '\r', ''), '\n', '')),
    '%Y-%m-%dt%H:%i:%sz'
);
"""
run(query)

In [9]:
query = """
ALTER TABLE observations
MODIFY `date` date;
"""
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()

In [10]:
select('observations')

,date,patient,encounter,description,value,units,type
0,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,body height,193.3,cm,numeric
1,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,pain severity - 0-10 verbal numeric rating [sc...,2.0,{score},numeric
2,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,body weight,87.8,kg,numeric
3,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,body mass index,23.5,kg/m2,numeric
4,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,diastolic blood pressure,82.0,mm[hg],numeric
